In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [2]:
lstm = nn.LSTM(3, 3) # Input dim is 3, output dim is 3
inputs = [torch.randn(1, 3) for _ in range(5)] #Make a sequence of length 5
print('INPUT: ')
print(inputs, end= '\n')
# Initialize the hidden state
hidden = (torch.randn(1, 1, 3),
          torch.randn(1, 1, 3))

for i in inputs:
    # Step through the sequence one element at a time
    # after each step, hidden contains the hidden state
    out, hidden = lstm(i.view(1, 1, -1), hidden)
    
    print('I View:')
    print(i.view(1, 1, -1))
    
    print('OUT:')
    print(out)

    print('HIDDEN:')
    print(hidden)


    print('------------------\n')

INPUT: 
[tensor([[-0.5525,  0.6355, -0.3968]]), tensor([[-0.6571, -1.6428,  0.9803]]), tensor([[-0.0421, -0.8206,  0.3133]]), tensor([[-1.1352,  0.3773, -0.2824]]), tensor([[-2.5667, -1.4303,  0.5009]])]
I View:
tensor([[[-0.5525,  0.6355, -0.3968]]])
OUT:
tensor([[[-0.2682,  0.0304, -0.1526]]])
HIDDEN:
(tensor([[[-0.2682,  0.0304, -0.1526]]]), tensor([[[-1.0766,  0.0972, -0.5498]]]))
------------------

I View:
tensor([[[-0.6571, -1.6428,  0.9803]]])
OUT:
tensor([[[-0.5370,  0.0346, -0.1958]]])
HIDDEN:
(tensor([[[-0.5370,  0.0346, -0.1958]]]), tensor([[[-1.1552,  0.1214, -0.2974]]]))
------------------

I View:
tensor([[[-0.0421, -0.8206,  0.3133]]])
OUT:
tensor([[[-0.3947,  0.0391, -0.1217]]])
HIDDEN:
(tensor([[[-0.3947,  0.0391, -0.1217]]]), tensor([[[-1.0727,  0.1104, -0.2179]]]))
------------------

I View:
tensor([[[-1.1352,  0.3773, -0.2824]]])
OUT:
tensor([[[-0.1854,  0.0740, -0.0979]]])
HIDDEN:
(tensor([[[-0.1854,  0.0740, -0.0979]]]), tensor([[[-1.0530,  0.1836, -0.1731]]]))


In [3]:
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3)) # CLean out hidden state
out, hidden = lstm(inputs, hidden)
print(out)
print(hidden)

tensor([[[-0.0187,  0.1713, -0.2944]],

        [[-0.3521,  0.1026, -0.2971]],

        [[-0.3191,  0.0781, -0.1957]],

        [[-0.1634,  0.0941, -0.1637]],

        [[-0.3368,  0.0959, -0.0538]]])
(tensor([[[-0.3368,  0.0959, -0.0538]]]), tensor([[[-0.9825,  0.4715, -0.0633]]]))


# Example: An LSTM for Part-of-Speech Tagging

In [15]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]

word_to_ix = {}
for sent, tags in training_data:
    print(sent)
    print('\n')
    print( tags)
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)
tag_to_ix = {"DET" : 0, "NN": 1, "V": 2}

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

['The', 'dog', 'ate', 'the', 'apple']


['DET', 'NN', 'V', 'DET', 'NN']
['Everybody', 'read', 'that', 'book']


['NN', 'V', 'DET', 'NN']
{'that': 7, 'ate': 2, 'apple': 4, 'book': 8, 'read': 6, 'Everybody': 5, 'dog': 1, 'The': 0, 'the': 3}


In [5]:
training_data

[(['The', 'dog', 'ate', 'the', 'apple'], ['DET', 'NN', 'V', 'DET', 'NN']),
 (['Everybody', 'read', 'that', 'book'], ['NN', 'V', 'DET', 'NN'])]

# Create The Model

In [10]:
class LSTMTagger(nn.Module):
    
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
        
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        
        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()
    
    def init_hidden(self):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (torch.zeros(1, 1, self.hidden_dim),
                torch.zeros(1, 1, self.hidden_dim))
    
    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, self.hidden = self.lstm(
                    embeds.view(len(sentence), 1, -1), self.hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim= 1)
        return tag_scores

# Train The Model

In [17]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_func = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr= 0.1)


# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print('Scores before training')
    print(tag_scores)
    
# again, normally you would NOT do 300 epochs, it is toy data    
for epoch in range(300):
    for sentence, tags in training_data:
        #Step 1: Accumulates Gradients
        # We need to clear them out before each instance
        model.zero_grad()
        
        #Also, we need to clear out the hidden state of the LSTM,
        # Detaching it from its history on the las instance
        model.hidden = model.init_hidden()
        
        #Step 2: Get our inputs ready for the network, that is, 
        #turn them into Tensor of word indices
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)
        
        # Step 3: Run our forward pass
        tag_scores = model(sentence_in)
        
        # Step 4: Compute the loss, gradients, and update the
        # params by calling optimizer.step()
        loss = loss_func(tag_scores, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    
    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print('\nScores after training')
    print(tag_scores)    
        

Scores before training
tensor([[-1.3226, -0.8678, -1.1594],
        [-1.3887, -0.8316, -1.1543],
        [-1.3817, -0.8192, -1.1775],
        [-1.1885, -0.9154, -1.2209],
        [-1.2450, -0.8572, -1.2458]])

Scores after training
tensor([[-0.0590, -3.1675, -4.1870],
        [-5.8233, -0.0118, -4.7413],
        [-3.3344, -4.0328, -0.0548],
        [-0.0161, -4.8020, -4.8530],
        [-5.4424, -0.0119, -4.8920]])
